# Terragon.de Image Classifier (TIC-Torch)
### based on a PyTorch Convolutional Neural Network

## Why TIC-Torch?

The TIC-Torch application offers a convenient, fast and effective workflow to create, train and deploy individual and versatile image classifiers.

## What's the workflow?

1. Collect many jpg files of the objects you want to classify and arrange them in the "raw" directory. Create one subfolder for each object you want to classify and put the images inside. Example:
    - raw/dogs
    - raw/cats
    - raw/humans
3. Run TIC-Torch to resize and vary the images and to train the Neural Network.
4. Run TIC-Torch to classify a new jpg.


# Import OS and Image manipulation libraries

In [ ]:
# System Libraries

import os, sys
from distutils.dir_util import copy_tree
from PIL import Image, ImageEnhance
import PIL.ImageOps
import random

# PyTorch Libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

# Other Libraries we'll use
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

print("Libraries imported - ready to use PyTorch", torch.__version__)

# Define Variables

In [ ]:
path_images_raw = "raw/"
path_images_train = "train/"

resize_x = 128
resize_y = 128

batch_size = 50
shuffle = True

epochs = 100

# list subdirs in raw/

In [ ]:
path_images_raw_subdirs = os.listdir(path_images_raw)
print("raw/ subdirs: ",path_images_raw_subdirs)

# copy all images from raw/ to train/ 

In [ ]:
def ImagesCopy(path_images_raw,path_images_train):
    copy_tree(path_images_raw, path_images_train)
    path_images_train_subdirs = os.listdir(path_images_train)
    print("copy from raw to train/ folder. done.")
    print("train subdirs: ",path_images_train_subdirs)
ImagesCopy(path_images_raw,path_images_train)

# resize all images in train/ to neural network size 

In [ ]:
def ImagesResize():
    for subdir, dirs, files in os.walk(path_images_train):
        for file in files:
            path_image = os.path.join(subdir, file)
            if os.path.isfile(path_image):
                im = Image.open(path_image)
                f, e = os.path.splitext(path_image)
                imResize = im.resize((resize_x,resize_y), Image.Resampling.LANCZOS)
                imResize.save(f + '_resized.jpg', 'JPEG', quality=90)
                os.remove(path_image)
    print("resized all images in folder train/")
ImagesResize()

# create variations of all images in train/ 

In [ ]:
def ImageVariations():
    for subdir, dirs, files in os.walk(path_images_train):
        for file in files:
            path_image = os.path.join(subdir, file)
            if os.path.isfile(path_image):
                im = Image.open(path_image)
                
                f, e = os.path.splitext(path_image)
                enhancer = ImageEnhance.Brightness(im)
                imLighter = enhancer.enhance(1.8)
                imLighter.save(f + '_lighter.jpg', 'JPEG', quality=90)
                imDarker = enhancer.enhance(0.5)
                imDarker.save(f + '_darker.jpg', 'JPEG', quality=90)
                imInverted = PIL.ImageOps.invert(im)
                imInverted.save(f + '_inverted.jpg', 'JPEG', quality=90)
                randomRotate = random.randrange(-45,45)
                imRotated = im.rotate(randomRotate)
                imRotated.save(f + '_rotated.jpg', 'JPEG', quality=90)
                imMirror = im.transpose(Image.FLIP_LEFT_RIGHT)
                imMirror.save(f + '_mirrored.jpg', 'JPEG', quality=90)
                imGrey = im.convert('L')
                imGrey.save(f + '_greyscale.jpg', 'JPEG', quality=90)
    print("added a mirror and greyscale images for all images in folder train/")
ImageVariations()

# Transfer all Images to Tensors

In [ ]:
# Function to ingest data using training and test loaders
# Now load the images from the train folder

def load_dataset(path_images_train):
    # Load all of the images
    transformation = transforms.Compose([
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=path_images_train,
        transform=transformation
    )
    
    
    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=shuffle
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=shuffle
    )
        
    return train_loader, test_loader


# Get the class names
classes = os.listdir(path_images_train)
classes.sort()
classes_amount = len(classes)
print(len(classes), 'classes:')
print(classes)

# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(path_images_train)

# Define the Neural Net Model

In [ ]:
# Create a neural net class
class Net(nn.Module):
    # Constructor
    def __init__(self, num_classes=classes_amount):
        super(Net, self).__init__()
        
        # Our images are RGB, so input channels = 3. wW'll apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # We'll apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A second convolutional layer takes 12 input channels, and generates 12 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A third convolutional layer takes 12 inputs and generates 24 outputs
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # So our feature tensors are now 32 x 32, and we've generated 24 of them
        # We need to flatten these to map them to  the probability for each class
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=num_classes)

    def forward(self, x):
        # Use a relu activation function after layer 1 (convolution 1 and pool)
        x = F.relu(self.pool(self.conv1(x)))
        # Use a relu activation function after layer 1 (convolution 2 and drop)
        
        # Use a relu activation function after layer 3 (convolution 3)
        x = F.relu(self.pool(self.conv2(x)))
        
        # Drop some features after the 3rd convolution to prevent overfitting
        x = F.relu(self.drop(self.conv3(x)))
        # Only drop the features if this is a training pass
        x = F.dropout(x, training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        x = self.fc(x)
        # Return class probabilities via a softmax function 
        return F.log_softmax(x, dim=1)
    
print("CNN model class defined!")

# Train and Test the model

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Use the CPU or GPU as appropriate
        data, target = data.to(device), target.to(device)
        # Reset the optimizer
        optimizer.zero_grad()
        # Push the data forward through the model layers
        output = model(data)
        # Get the loss
        loss = loss_criteria(output, target)
        # Keep a running total
        train_loss += loss.item()
        # Backpropagate
        loss.backward()
        optimizer.step()
        # Print metrics for every 10 batches so we see some progress
        if batch_idx % 10 == 0:
            print('Training set [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    # return average loss for the epoch
    return train_loss / len(train_loader.dataset)
            
            
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # Get the predicted classes for this batch
            output = model(data)
            # calculate the loss and successful predictions for this batch
            test_loss += loss_criteria(output, target).item()
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calculate the average loss and total accuracy for this epoch
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return test_loss
    
    
# Now use the train and test functions to train and test the model    

device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"
print('Training on', device)

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=len(classes)).to(device)

# Use an "Adam" optimizer to adjust weights
# (see https://pytorch.org/docs/stable/optim.html#algorithms for details of supported algorithms)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.CrossEntropyLoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []

# Train over 5 epochs (in a real scenario, you'd likely use many more)
epochs = epochs

if os.path.isfile("model.pt"):
    print("model.pt already exists and is now loaded, no training!")
    model = torch.load("model.pt")
else:
    for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)
    %matplotlib inline
    from matplotlib import pyplot as plt

    plt.plot(epoch_nums, training_loss)
    plt.plot(epoch_nums, validation_loss)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()
    torch.save(model, "model.pt")

# Evaluate the training - Confusion Matrix

In [ ]:
#Pytorch doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn
from sklearn.metrics import confusion_matrix

# Set the model to evaluate mode
model.eval()

# Get predictions for the test data and convert to numpy arrays for use with SciKit-Learn
print("Getting predictions from test set...")
truelabels = []
predictions = []
for data, target in test_loader:
    for label in target.cpu().data.numpy():
        truelabels.append(label)
    for prediction in model.cpu()(data).data.numpy().argmax(1):
        predictions.append(prediction) 

# Plot the confusion matrix
cm = confusion_matrix(truelabels, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Greys)
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel("Predicted Shape")
plt.ylabel("True Shape")
plt.show()

# Predict a new image

In [ ]:
# Function to predict the class of an image
def predict_image(classifier, image):
    import numpy
    
    # Set the classifer model to evaluation mode
    classifier.eval()
    
    # Apply the same transformations as we did for the training images
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Preprocess the image
    image_tensor = transformation(image).float()

    # Add an extra batch dimension since pytorch treats all inputs as batches
    image_tensor = image_tensor.unsqueeze_(0)

    # Turn the input into a Variable
    input_features = Variable(image_tensor)

    # Predict the class of the image
    output = classifier(input_features)
    index = output.data.numpy().argmax()
    return index


#Now let's try it with a new image
from random import randint
from PIL import Image
import os, shutil
    
imgFile = Image.open("classify.jpg")
imgFile = imgFile.resize((resize_x,resize_y), Image.Resampling.LANCZOS)

# Display the image
plt.imshow(imgFile)

# Call the predction function
index = predict_image(model, imgFile)
print(classes[index])

In [ ]:
print("Visit http://www.Terragon.de")